In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset

In [65]:
# Define your dataset (assuming it's in a CSV file)
data_file = "datatraining_new.csv"  # Update with your file path
data = pd.read_csv(data_file)

# Extract columns to normalize
columns_to_normalize = data.columns.drop("Occupancy")

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the selected columns
data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])

# Extract features (X) and target (y)
X = data.drop("Occupancy", axis=1).values
y = data["Occupancy"].values


# Convert X and y to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Create a PyTorch TensorDataset
train_dataset = TensorDataset(X_tensor, y_tensor)



In [76]:
# Define the Liquid Time Constant (LTC) network model
class LTCNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LTCNetwork, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize weights and biases
        self.W_in = nn.Parameter(torch.randn(hidden_size, input_size))
        self.W_rec = nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.W_out = nn.Linear(hidden_size, output_size)
        self.tau = 1.0  # Time constant

    def forward(self, x):
        batch_size, seq_len, input_dim = x.size()
        hidden_state = torch.zeros(batch_size, self.hidden_size).to(x.device)

        # Compute hidden state dynamics
        for t in range(seq_len):
            input_t = x[:, t, :]  # Select input at time step t
            hidden_state = (1 - 1 / self.tau) * hidden_state + torch.tanh(self.W_in @ input_t.t() + self.W_rec @ hidden_state.unsqueeze(2)).squeeze(2)

        # Compute output using the last hidden state
        output = self.W_out(hidden_state)
        return output


In [80]:
# Set hyperparameters
input_size = X_tensor.shape[1]  # Input size based on the number of features
hidden_size = 64
output_size = 1
num_epochs = 10
learning_rate = 0.001
batch_size = 64


# Initialize the LTC network model
model = LTCNetwork(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # For binary classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create data loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs = inputs.unsqueeze(1)  # Add a singleton dimension for sequence length
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Reshape labels to match the shape of outputs
        labels = labels.view(-1, outputs.size(1))  # Reshape labels to match the shape of outputs
        
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

ValueError: Target size (torch.Size([1, 64])) must be the same as input size (torch.Size([64, 64]))